In [43]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import Sequence
from tensorflow.keras import Input, regularizers, Sequential, optimizers, preprocessing, initializers
from tensorflow.keras.activations import relu, tanh
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dense, BatchNormalization, Dropout, Flatten, InputLayer, Conv2D, Activation
from tensorflow import config
from tensorflow.keras.applications import VGG16
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from math import ceil
import pandas as pd
import numpy as np
from skimage import io
import os
from sklearn.model_selection import train_test_split
print("GPU is", "available" if config.list_physical_devices('GPU') else "NOT AVAILABLE")

GPU is available


In [2]:
PATH = os.path.join("dataset", "data_augmented")

In [54]:
def create_model(filters, learning_rate=0.2, activation="relu", input_shape=(400, 400, 1)):
    model = Sequential()
    model.add(Conv2D(filters[0],  kernel_size = (5, 5), input_shape=input_shape, 
                     kernel_initializer=initializers.GlorotNormal(), padding="same"))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(BatchNormalization())
    model.add(Activation(activation)) 
    
    for index, filter_ in enumerate(filters[1:-2]):
        model.add(Conv2D(filter_, kernel_size = (3, 3), activation=activation, padding="same"))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(BatchNormalization())
        model.add(Activation(activation)) 

    model.add(Flatten())
    model.add(Dense(filters[-2]))
    model.add(BatchNormalization())
    model.add(Activation(activation))  
    model.add(Dropout(0.3))
    
    model.add(Dense(filters[-1]))
    model.add(BatchNormalization())
    model.add(Activation(activation)) 
    model.add(Dropout(0.2))
    
    model.add(Dense(4, activation = 'softmax'))
    
    opt = optimizers.Adam(learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opt,
                 metrics=['categorical_accuracy'])
    
    return model

In [55]:
class BatchGenerator(Sequence) :
  
        def __init__(self, x_set, y_set, batch_size):
            self.x, self.y = x_set, y_set
            self.batch_size = batch_size

        def __len__(self):
            return ceil(len(self.x) / self.batch_size)

        def __getitem__(self, idx):
            batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
            batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

            return np.array([StandardScaler().fit_transform(io.imread(os.path.join(PATH, file_name))).reshape((400, 400, 1))             
                   for file_name in batch_x]), np.array(batch_y)

In [56]:
df = pd.read_csv("dataset/name_data_augmented.csv", index_col=0)
df.head()

,filename,label
0,PROESTRO_535_1_0_1_fliped.jpg,PROESTRO
1,ESTRO_227_2_90_2_fliped.jpg,ESTRO
2,METAESTRO_153_3_90_0_fliped.jpg,METAESTRO
3,DIESTRO_603_4_90_0_fliped.jpg,DIESTRO
4,ESTRO_547_2_0_1_fliped.jpg,ESTRO


In [57]:
X = df.filename.values
Y = df.label.values
x, x_test, y, y_test = train_test_split(X, Y, test_size=0.1)

In [58]:
kf = StratifiedKFold(n_splits=4, shuffle=True)
batch_size = 32
loss_list = []
categorical_accuracy_list = []

for train_index, val_index in kf.split(x, y):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = pd.get_dummies(y[train_index]).values, pd.get_dummies(y[val_index]).values
    
    train_batch_generator = BatchGenerator(x_train, y_train, batch_size=batch_size)
    val_batch_generator = BatchGenerator(x_val, y_val, batch_size=batch_size)
    
    model = create_model(filters=[8, 16, 32, 32, 64, 64, 16], learning_rate=0.005, activation="relu")
    model.fit(x=train_batch_generator, verbose=1, epochs=5)
    loss, categorical_accuracy = model.evaluate(x=val_batch_generator, verbose=0)
    
    print("Val loss:{}  | Val accuracy:{}".format(loss, categorical_accuracy))
    print("*******************************************************************")

Epoch 1/5
337/337 [==============================] - 102s 304ms/step - loss: 0.5188 - categorical_accuracy: 0.8045
Epoch 2/5
337/337 [==============================] - 85s 253ms/step - loss: 0.2164 - categorical_accuracy: 0.9252
Epoch 3/5
337/337 [==============================] - 86s 254ms/step - loss: 0.1517 - categorical_accuracy: 0.9499
Epoch 4/5
337/337 [==============================] - 86s 254ms/step - loss: 0.1216 - categorical_accuracy: 0.9575
Epoch 5/5
337/337 [==============================] - 86s 255ms/step - loss: 0.0837 - categorical_accuracy: 0.9729
Val loss:0.20173659920692444  | Val accuracy:0.9403234720230103
*******************************************************************
Epoch 1/5
337/337 [==============================] - 85s 253ms/step - loss: 0.5507 - categorical_accuracy: 0.7855
Epoch 2/5
337/337 [==============================] - 85s 253ms/step - loss: 0.2533 - categorical_accuracy: 0.9173
Epoch 3/5
337/337 [==============================] - 86s 254ms/step -

In [61]:
test_generator = BatchGenerator(x_test, pd.get_dummies(y_test).values, batch_size=batch_size)
model.evaluate(test_generator, verbose=1)

50/50 [==============================] - 18s 355ms/step - loss: 0.1079 - categorical_accuracy: 0.9617


[0.10785161703824997, 0.961731493473053]

In [62]:
model.save("model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model\assets
